In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.4MB/s 
     |████████████████████████████████| 901kB 42.6MB/s 
     |████████████████████████████████| 3.3MB 46.3MB/s 


In [2]:
!pip install datasets

     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 245kB 43.3MB/s 
     |████████████████████████████████| 122kB 52.7MB/s 


In [3]:
!pip install tokenizers

In [4]:
import transformers
import datasets
import tokenizers

for module in (transformers, datasets, tokenizers):
  print(module.__name__, module.__version__)

transformers 4.8.2
datasets 1.8.0
tokenizers 0.10.3


In [5]:
from transformers import pipeline

In [6]:
import inspect
inspect.signature(pipeline).parameters

mappingproxy({'config': <Parameter "config: Union[str, transformers.configuration_utils.PretrainedConfig, NoneType] = None">,
              'feature_extractor': <Parameter "feature_extractor: Union[str, ForwardRef('SequenceFeatureExtractor'), NoneType] = None">,
              'framework': <Parameter "framework: Union[str, NoneType] = None">,
              'kwargs': <Parameter "**kwargs">,
              'model': <Parameter "model: Optional = None">,
              'model_kwargs': <Parameter "model_kwargs: Dict[str, Any] = {}">,
              'revision': <Parameter "revision: Union[str, NoneType] = None">,
              'task': <Parameter "task: str">,
              'tokenizer': <Parameter "tokenizer: Union[str, transformers.tokenization_utils.PreTrainedTokenizer, NoneType] = None">,
              'use_auth_token': <Parameter "use_auth_token: Union[str, bool, NoneType] = None">,
              'use_fast': <Parameter "use_fast: bool = True">})

In [7]:
classifier = pipeline("sentiment-analysis")

In [8]:
classifier("I've been waiting for a HuggingFace course my whole life.")

[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [9]:
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!"
])

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [10]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint) #오토토크나이저 인스턴스화해서 토크나이저에 넣음

In [11]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [12]:
inputs["input_ids"]

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]])

In [13]:
inputs["input_ids"].size() #행렬화해야 이것을 넣을 수 있다.

torch.Size([2, 16])

### Going through the model

In [14]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
# 워닝.

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model #파이토치.. 허깅페이스 장점.. 형태를 보여줌..
# 워드임베딩, 포지션임베딩이 있네.

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [16]:
outputs = model(**inputs) ##딕셔너리 객체. 키와 벨루가 넘어감.  /리스트면 벨루가 넘어감.
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [17]:
inspect.signature(model.forward).parameters

mappingproxy({'attention_mask': <Parameter "attention_mask=None">,
              'head_mask': <Parameter "head_mask=None">,
              'input_ids': <Parameter "input_ids=None">,
              'inputs_embeds': <Parameter "inputs_embeds=None">,
              'output_attentions': <Parameter "output_attentions=None">,
              'output_hidden_states': <Parameter "output_hidden_states=None">,
              'return_dict': <Parameter "return_dict=None">})

In [18]:
model.__class__.__name__ #피처 익스트렉터??  786차원으로 추출 기본적인 모델..

'DistilBertModel'